In [13]:
import numpy as np
import pandas as pd
from predictor import KValuePredictor

In [14]:
file_path = r'C:\Users\deban\Desktop\Imp Docs\RaceCar\Dataset\RaceCarUsefulDataDriver1.xlsx' 
dft = pd.read_excel(file_path, sheet_name="Sheet1")  

In [15]:
slope = (65-30)/(0-125)
# hardness at 23deg C
H_C1 = 85
H_C2 = 80
H_C3 = 75
H_C4 = 70
H_C5 = 65
shoreA_H = np.array([H_C1, H_C2, H_C3, H_C4, H_C5])

# Tire temperatures
tire11 = dft["front_surface_temp"].to_numpy()
tire21 = dft["rear_surface_temp"].to_numpy()
tire31 = dft["front_inner_temp"].to_numpy()
tire41 = dft["rear_inner_temp"].to_numpy()

# Ext factors
humidity2 = dft["Humidity"].to_numpy()
temp_surr2 = dft["Ambient_Temperature"].to_numpy()
surface_rougness2 = dft["Surface_Roughness"].to_numpy()

# Car specific factors
force2 = dft["force_on_tire"].to_numpy()
tire_type2 = dft["Tire_Compound"].to_numpy()  # Keep as original categorical
fric_coeff2 = dft["Tire_Friction_Coefficient"].to_numpy()  # Added .to_numpy()
v2 = dft["Speed"].to_numpy()
t2 = dft["Tire_wear"].to_numpy()
n1 = len(t2)

# Calculate E values (your existing calculation code...)
H11 = np.zeros(n1)
H21 = np.zeros(n1)
H31 = np.zeros(n1)
H41 = np.zeros(n1)
H11_v = np.zeros(n1)
H21_v = np.zeros(n1)
H31_v = np.zeros(n1)
H41_v = np.zeros(n1)
E11 = np.zeros(n1)
E21 = np.zeros(n1)
E31 = np.zeros(n1)
E41 = np.zeros(n1)

tire_hardness_map1 = {'C1': shoreA_H[0], 'C2': shoreA_H[1], 'C3': shoreA_H[2], 'C4': shoreA_H[3], 'C5': shoreA_H[4]}

for i in range(n1):
    H0 = tire_hardness_map1[tire_type2[i]]
    temp_diff1 = np.array([tire11[i], tire21[i], tire31[i], tire41[i]]) - 23
    hardness_adjustments1 = H0 + slope * temp_diff1
    H11[i], H21[i], H31[i], H41[i] = hardness_adjustments1
    hardness_vickers1 = 10.363 * np.exp(0.0822 * hardness_adjustments1)
    H11_v[i],H21_v[i],H31_v[i],H41_v[i] = hardness_vickers1
    Emodulus1 = (0.0981*(56 + 7.66 * hardness_adjustments1)) / (0.137505 * (254 - 2.54*hardness_adjustments1))
    E11[i], E21[i], E31[i], E41[i] = Emodulus1

# Calculate maximum E values
E_new = np.zeros(n1)
for i in range(n1):
    E_new[i] = np.max([E11[i], E21[i], E31[i], E41[i]])
E_new = E_new * 10**(6)  # Convert to proper units

In [16]:
# Create DataFrame with correct feature names for the predictor
prediction_data = pd.DataFrame({
    'tire1': tire11,           # front_surface_temp -> tire1
    'tire2': tire21,           # rear_surface_temp -> tire2  
    'tire3': tire31,           # front_inner_temp -> tire3
    'tire4': tire41,           # rear_inner_temp -> tire4
    'humidity1': humidity2,     # Humidity -> humidity1
    'temp_surr1': temp_surr2,   # Ambient_Temperature -> temp_surr1
    'surface_rougness1': surface_rougness2,  # Surface_Roughness -> surface_rougness1
    'force1': force2,          # force_on_tire -> force1
    'fric_coeff1': fric_coeff2, # Tire_Friction_Coefficient -> fric_coeff1
    'v1': v2,                  # Speed -> v1
    't1': t2,                  # Tire_wear -> t1
    'tire_type_encoded1': tire_type2,  # Keep original categorical values
    'E': E_new                 # Calculated E values
})

In [19]:
# Initialize predictor
predictor = KValuePredictor()
predictor.get_model_info()

# Convert DataFrame to list of dictionaries for batch prediction
prediction_dicts = prediction_data.to_dict('records')

# Make predictions with progress tracking
predicted_K = []
print(f"Making predictions for {len(prediction_dicts)} samples...")

for i in range(len(prediction_dicts)):
    if i % 100 == 0:  # Print progress every 100 predictions
        print(f"Predicting sample {i+1}/{len(prediction_dicts)} ({(i+1)/len(prediction_dicts)*100:.1f}%)")
    
    # Use predict() for single samples, not predict_batch()
    k_value = predictor.predict(prediction_dicts[i])
    predicted_K.append(k_value)

print("Predictions completed!")

# Display the results
print(f"\nPredicted {len(predicted_K)} K values:")
for i, k_val in enumerate(predicted_K[:10]):  # Show first 10 predictions
    print(f"Sample {i+1}: K = {k_val:.6e}")

# Add predictions back to your original dataframe
dft['predicted_K'] = predicted_K
print(f"\nPredictions added to dataframe. Shape: {dft.shape}")


Model loaded from C:\\Users\\deban\\Desktop\\Imp Docs\\RaceCar\\k_predictor_model.keras
Scalers loaded successfully
K VALUE PREDICTION MODEL
Input features expected: 17
Model architecture: 6 layers
Ready to predict K values from unscaled inputs
Making predictions for 12099 samples...
Predicting sample 1/12099 (0.0%)


c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has

Predicting sample 101/12099 (0.8%)


c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has

Predicting sample 201/12099 (1.7%)


...

Predicting sample 11901/12099 (98.4%)


c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has

Predicting sample 12001/12099 (99.2%)


c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\base.py:486: UserWarning: X has

Predictions completed!

Predicted 12099 K values:
Sample 1: K = 2.863107e-10
Sample 2: K = 2.863107e-10
Sample 3: K = 2.863107e-10
Sample 4: K = 2.863107e-10
Sample 5: K = 2.863107e-10
Sample 6: K = 2.863107e-10
Sample 7: K = 2.863107e-10
Sample 8: K = 2.863107e-10
Sample 9: K = 2.863107e-10
Sample 10: K = 2.863107e-10

Predictions added to dataframe. Shape: (12099, 19)


we have K ready, H_v ready for every tyre, well estimate the wear of each tire, and sum of worn out material of each tire. When the estimated wearing time for 550g limiting wearing is less that 5 minutes, an alert of boxing the next lap should be displayed and also the estimated time remaining for the limiting wearing should be displayed.

access i from 1 to n and find wij = K[i] * F[i] * V[i] * 0.277778 * (t[i] - t[i-1])/ H_vj[i] for jth tire sumj+= wij time_passed += (t[i] - t[i-1]) time_rem = W_lim * time_passed / max(sumj) - time_passed if (time_rem< 8 * 60) { print(box next lap) }

In [20]:
# Initialize variables
sumW = np.zeros(4)
time_rem = np.zeros(4)
K1 = np.array(predicted_K).flatten()  # Ensure it's a flat numpy array
H = np.array([H11_v, H21_v, H31_v, H41_v])  # Shape: (4, n1)
F = force2
V = v2
T = dft["lap_time"].to_numpy()
time_elapsed = 0
W_lim = 700  # grams

print(f"Starting tire wear monitoring...")
print(f"K1 shape: {K1.shape}, H shape: {H.shape}")
print(f"Data length: {n1}")

# Main loop with safety checks
for i in range(1, min(n1, len(K1))):
    try:
        for j in range(4):  # Iterate over the 4 tires
            if i < H.shape[1]:  # Safety check for H dimensions
                wear_increment = (K1[i] * F[i] * V[i] * 0.27777778 * 
                                (T[i] - T[i - 1]) * 1.1) * 10**6 / H[j, i]
                sumW[j] += wear_increment
        
        time_elapsed += T[i] - T[i - 1]
        
        # Avoid division by zero
        safe_sumW = np.where(sumW > 0, sumW, 1e-10)
        time_rem = ((W_lim / safe_sumW) - 1) * time_elapsed
        
        if i % 10 == 0:  # Print every 10th iteration to avoid spam
            print(f"Lap {i}: Max wear = {np.max(sumW):.2f}g, Min time remaining = {np.min(time_rem):.1f}s")
        
        if np.any(time_rem <= 8 * 60):  # 8 minutes
            print("🚨 BOX NEXT LAP! 🚨")
            print(f"Time remaining = {np.min(time_rem):.1f} seconds")
            print(f"Maximum tire weight lost = {np.max(sumW):.2f} g")
            print(f"Critical tire: {np.argmax(sumW) + 1}")
            break
            
    except Exception as e:
        print(f"Error at iteration {i}: {e}")
        break

print("Tire wear monitoring complete.")


Starting tire wear monitoring...
K1 shape: (12099,), H shape: (4, 12099)
Data length: 12099
Lap 10: Max wear = 0.76g, Min time remaining = 10144.5s
Lap 20: Max wear = 1.78g, Min time remaining = 8632.4s
Lap 30: Max wear = 2.98g, Min time remaining = 7493.5s
Lap 40: Max wear = 6.95g, Min time remaining = 4188.0s
Lap 50: Max wear = 12.26g, Min time remaining = 3140.1s
Lap 60: Max wear = 14.67g, Min time remaining = 3224.4s
Lap 70: Max wear = 17.38g, Min time remaining = 3142.4s
Lap 80: Max wear = 24.20g, Min time remaining = 2569.7s
Lap 90: Max wear = 27.13g, Min time remaining = 2529.8s
Lap 100: Max wear = 29.88g, Min time remaining = 2512.0s
Lap 110: Max wear = 33.55g, Min time remaining = 2443.1s
Lap 120: Max wear = 43.05g, Min time remaining = 2121.4s
Lap 130: Max wear = 49.99g, Min time remaining = 1963.6s
Lap 140: Max wear = 62.04g, Min time remaining = 1717.2s
Lap 150: Max wear = 68.09g, Min time remaining = 1642.6s
Lap 160: Max wear = 75.38g, Min time remaining = 1591.0s
Lap 170: